In [1]:
import pandas as pd

In [2]:
text_data = pd.read_excel('/Users/rii_beltz/Desktop/candidate_classifier/data/processed/processed_data.xlsx')


обработка текста:
- удаление лишних символов (пунктуация, спецсимволы)
- приведение текста к нижнему регистру
- токенизация
- удаление стоп-слов
- лемматизация

In [3]:
import ssl
import nltk

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rii_beltz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/rii_beltz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /Users/rii_beltz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
import nltk
print(nltk.data.path)

nltk.download('punkt', download_dir='/Users/rii_beltz/nltk_data')
nltk.download('stopwords', download_dir='/Users/rii_beltz/nltk_data')
nltk.data.path.append('/Users/rii_beltz/nltk_data')


['/Users/rii_beltz/nltk_data', '/Users/rii_beltz/Desktop/candidate_classifier/venv/nltk_data', '/Users/rii_beltz/Desktop/candidate_classifier/venv/share/nltk_data', '/Users/rii_beltz/Desktop/candidate_classifier/venv/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data']


[nltk_data] Downloading package punkt to /Users/rii_beltz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rii_beltz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/rii_beltz/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [7]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Пример текста
text = "This is a simple test sentence for tokenization and stopwords removal."

stop_words = set(stopwords.words('english'))  # Загружаем стоп-слова
tokens = word_tokenize(text)  # Токенизация
filtered_tokens = [word for word in tokens if word.lower() not in stop_words]  # Удаление стоп-слов

print("Токены:", tokens)
print("После удаления стоп-слов:", filtered_tokens)


Токены: ['This', 'is', 'a', 'simple', 'test', 'sentence', 'for', 'tokenization', 'and', 'stopwords', 'removal', '.']
После удаления стоп-слов: ['simple', 'test', 'sentence', 'tokenization', 'stopwords', 'removal', '.']


In [8]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import re

def preprocess_text(text):
    # Очистка текста
    text = re.sub(r'\s+', ' ', text)  # Удаление лишних пробелов
    text = re.sub(r'[^\w\s]', '', text)  # Удаление пунктуации
    text = text.lower()  # Приведение к нижнему регистру

    # Токенизация
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('russian'))  # Загрузка стоп-слов
    tokens = [word for word in tokens if word not in stop_words]

    return tokens


In [9]:
import re

def extract_experience(text):
    pattern = r"(опыт работы (от|до)? ?\d+ (год|лет|месяц|года|годы)?)|(без опыта работы)"
    matches = re.findall(pattern, text.lower())
    return [match[0] for match in matches if match[0]]


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

# Загрузить стоп-слова для русского языка
stop_words_russian = stopwords.words('russian')

# Инициализация CountVectorizer с пользовательским списком стоп-слов
vectorizer = CountVectorizer(ngram_range=(2, 3), stop_words=stop_words_russian, max_features=50)

# Применение к столбцу с описанием вакансий
ngrams = vectorizer.fit_transform(text_data['Описание вакансии'])
ngrams_features = vectorizer.get_feature_names_out()

# Вывод найденных n-грамм
print(ngrams_features)


['16 лет' 'вакансия жду' 'вакансия жду именно' 'вносить свои идеи'
 'встречи собеседовании' 'гражданско правового'
 'гражданско правового характера' 'график работы' 'данная вакансия'
 'данная вакансия жду' 'договору гражданско'
 'договору гражданско правового' 'доп мотивацию' 'доп мотивацию первые'
 'жду именно' 'жду именно твой' 'заинтересовала данная'
 'заинтересовала данная вакансия' 'заявки подключение'
 'идеи реализации работе' 'именно твой' 'именно твой отклик'
 'индивидуальный график' 'индивидуальный график работы'
 'клиентов действующим тарифам' 'который сможешь расти' 'месяца работы'
 'мобильная связь' 'мотивацию первые' 'мотивацию первые месяца'
 'наставника который' 'опыта работы' 'отклик встречи'
 'отклик встречи собеседовании' 'отчетность требования' 'оформлять заявки'
 'первые месяца' 'первые месяца работы' 'подключение услуг'
 'правового характера' 'свои идеи' 'свои идеи реализации'
 'совмещение учебой' 'твой отклик' 'твой отклик встречи'
 'трудоустройство договору' 'тру

In [18]:
from transformers import pipeline

# Загрузка альтернативной модели
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Классификация текста
categories = ["обязанности", "требования", "условия"]
text = "Обязанности: Продвигать услуги компании. Требования: Желание зарабатывать."
result = classifier(text, categories)
print(result)


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBartForSequenceClassification: ['model.decoder.version', 'model.encoder.version']
- This IS expected if you are initializing TFBartForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBartForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBartForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForSequenceClassification for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use 0


{'sequence': 'Обязанности: Продвигать услуги компании. Требования: Желание зарабатывать.', 'labels': ['обязанности', 'условия', 'требования'], 'scores': [0.5210112929344177, 0.25631389021873474, 0.22267486155033112]}


In [24]:
import re

# Функция для извлечения опыта работы с помощью регулярных выражений
def extract_experience_with_regex(text):
    # Регулярное выражение для поиска опыта работы
    experience_patterns = [
        r"\b(\d+)\s*(год[а-я]*)",  # Например, "2 года"
        r"\b(\d+)\s*(месяц[а-я]*)",  # Например, "6 месяцев"
        r"без\sопыта\sработы"  # Упоминание "без опыта работы"
    ]
    
    experience_phrases = []
    for pattern in experience_patterns:
        matches = re.findall(pattern, text, re.IGNORECASE)
        for match in matches:
            if isinstance(match, tuple):
                experience_phrases.append(" ".join(match).strip())
            else:
                experience_phrases.append(match.strip())
    
    return experience_phrases

# Пример текста вакансии
vacancy_text = """
'Что мы предлагаем:  Доход до 80000 ₽, состоит из оклада и ежемесячной премии по результатам работы; График работы 5/2; Оформление в штат с первого рабочего дня, расширенный соцпакет для опытных сотрудников: компенсация затрат на фитнес или медицинские услуги, дополнительные выплаты к отпуску, компенсация процентов по ипотечному кредиту; Оборудование и спецодежда предоставляются, компенсация за использование личного автомобиля-при наличии; Всестороннее развитие: корпоративный онлайн-университет, доступ к электронным библиотекам с широким выбором бизнес-литературы, компенсация профильного внешнего обучения; Компенсация сотовой связи и специальные условия на услуги компании для сотрудников, корпоративные скидки у компаний-партнеров.  Чем предстоит заниматься:  Обслуживание текущих абонентов (физических лиц) и устранение перебоев доступа к услугам компании: интернет, интерактивное ТВ, видеонаблюдение, умный дом; Ремонт линий связи и настройка оборудования, проверка работоспособности; Консультирование клиентов по работе оборудования и техническим вопросам; Ведение отчетности в мобильном приложении.  Что мы от вас ждем:  Готовность к разъездному и интенсивному характеру работы; Грамотная речь Быстрая обучаемость и стремление постоянно учиться новому; Опыт работы в телекоммуникациях (подключение услуг связи или выездная техническая поддержка) Будет вашим преимуществом.  Готовы рассмотреть кандидатов с опытом работы на вакансиях: техник, монтажник, электрик, электромонтёр, системный администратор, техник-слаботочник, выездной мастер, инженер-монтажник, сервисный инженер.'
"""

# Извлечение опыта работы
experience = extract_experience_with_regex(vacancy_text)
print("Извлеченный опыт работы:", experience)


Извлеченный опыт работы: []
